<a href="https://colab.research.google.com/github/Gautam0507/ECE034-Assignments/blob/main/Assignment-1/ECE304_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook By:

Name: N Gautam

Roll_No: 2210110411

Name: Risshi Kamalesh K

Roll No: 2210110511

# Assignment 1 ECE304 - AIML

Implementing a Bayesian Classifier for the given data

# Importing and Installing Python libraries

In [ ]:
%%capture
!pip install pandas numpy scipy

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Importing the data into the notebook

In [ ]:
%%capture
!curl -o Assignment-1.csv https://raw.githubusercontent.com/Gautam0507/ECE034-Assignments/refs/heads/main/Assignment-1/Assignment-1.csv

In [ ]:
!ls

Assignment-1.csv  sample_data


In [ ]:
data = pd.read_csv('Assignment-1.csv')

# Defining given Constants and Data

In [ ]:
means = {
    0: (-1, -1),
    1: (-1, 1),
    2: (1, -1),
    3: (1, 1)
}

# Same value defined multiple times for genralizing the bayesian function
variances = {
    0: (0.1 , 0.1),
    1: (0.1 , 0.1),
    2: (0.1 , 0.1),
    3: (0.1 , 0.1)
}

priors  = {
    0: 0.1,
    1: 0.3,
    2: 0.25,
    3: 0.35
}

In [ ]:
X = data[['x1', 'x2']].values
y_true = data['y'].values

In [ ]:
print(f"Dimensions of X: {X.shape}")
print(f"Dimensions of y: {y_true.shape}")

Dimensions of X: (10000, 2)
Dimensions of y: (10000,)


# Defining Functions

In [ ]:
# The likelihood function
def gaussian_pdf(x, mean, variance):
  return (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-(x - mean) ** 2 / (2 * variance))

In [ ]:
def bayesian_classifier(x, priors, feature_means, feature_variances):
    posteriors = []
    for class_label in range(len(priors)):
        # Calculate likelihood for each feature
        likelihood = 1
        for i in range(len(x)):
            likelihood *= gaussian_pdf(x[i], feature_means[class_label][i], feature_variances[class_label][i])

        # Calculate the posterior for this class
        posterior = likelihood * priors[class_label]
        posteriors.append(posterior)

    # Predict the class with the maximum posterior
    return np.argmax(posteriors)

# Part (a)

# Making predictions for all samples

In [ ]:
y_pred = np.array([bayesian_classifier(x, priors, means, variances) for x in X])

# Calculating Probability of Error

In [ ]:
error_count = np.sum(y_true != y_pred)
probability_of_error = error_count / len(y_true)

print(f"Probability of Classification Error: {probability_of_error:.4f}")

Probability of Classification Error: 0.5956


# Part (b)

# Splitting the data into training and testing samples

In [ ]:
train_data = data.iloc[:5000]
test_data = data.iloc[5000:]

In [ ]:
X_test = test_data[['x1', 'x2']].values
y_true_test = test_data['y'].values

# Estimating Prior Probabilities, means and variances from first 5000 samples

In [ ]:
class_counts = train_data['y'].value_counts()
prior_estimates = class_counts / len(train_data)

In [ ]:
feature_variances = {}
feature_means = {}

In [ ]:
for class_label in prior_estimates.keys():
  class_data = train_data[train_data['y']== class_label]
  feature_means[class_label] = class_data[['x1', 'x2']].mean().values
  feature_variances[class_label] = (np.var(class_data['x1']), np.var(class_data['x2']))

In [ ]:
feature_variances = {key: feature_variances[key] for key in sorted(feature_variances)}
feature_means = {key: feature_means[key] for key in sorted(feature_means)}

# Make predictions using estimated values

In [ ]:
y_pred_test_initial = np.array([bayesian_classifier(x, priors, means, variances) for x in X_test])
y_pred_test_estimate = np.array([bayesian_classifier(x, prior_estimates, feature_means, feature_variances) for x in X_test])

# Calculating the probability of error

In [ ]:
error_initial = np.mean(y_pred_test_initial != y_true_test)
error_estimated = np.mean(y_pred_test_estimate != y_true_test)

In [ ]:
print(f"Initial Bayesian Classifier Error: {error_initial:.4f}")
print(f"Updated Bayesian Classifier Error: {error_estimated:.4f}")
print(f"Estimated Feature Variances:       {feature_variances}")
print(f"Estimated Feature Means:           {feature_means}")
print(f"Estimated Prior Probabilities:     {prior_estimates}")

Initial Bayesian Classifier Error: 0.5860
Updated Bayesian Classifier Error: 0.0018
Estimated Feature Variances:       {0.0: (0.10560770123046481, 0.09543178034780116), 1.0: (0.10218427264247697, 0.09675263966564658), 2.0: (0.09668160800212369, 0.09771636937331371), 3.0: (0.09680346510321135, 0.10061416195818157)}
Estimated Feature Means:           {0.0: array([-1.02994496, -1.01138844]), 1.0: array([-1.01143664,  1.00047817]), 2.0: array([0.99457962, 1.00154235]), 3.0: array([ 1.00255154, -1.00503952])}
Estimated Prior Probabilities:     y
3.0    0.3480
1.0    0.2926
2.0    0.2560
0.0    0.1034
Name: count, dtype: float64
